# Stat Arb Deploy Tool Demo

This notebook demonstrates the usage of the Stat Arb Deploy Tool methods.

In [ ]:
import logging
import os
import time
from datetime import datetime
from decimal import Decimal
from unittest.mock import patch
from dotenv import load_dotenv
import pandas as pd
import plotly.graph_objects as go
from typing import Dict, Any

logging.basicConfig(level=logging.INFO)
logging.getLogger("asyncio").setLevel(logging.CRITICAL)
load_dotenv()

from hummingbot.strategy.script_strategy_base import ScriptStrategyBase
from hummingbot.strategy_v2.executors.grid_executor.data_types import GridExecutorConfig
from hummingbot.strategy_v2.executors.grid_executor.grid_executor import GridExecutor
from hummingbot.strategy_v2.executors.position_executor.data_types import TripleBarrierConfig, TrailingStop
from plotly.subplots import make_subplots

from core.data_sources.clob import CLOBDataSource
from core.services.mongodb_client import MongoDBClient
from core.services.backend_api_client import BackendAPIClient


async def create_coint_figure(connector_instance,
                              controller_config,
                              base_candles,
                              quote_candles,
                              extra_info,
                              plot_prices: bool = False):
    fig = make_subplots(
        rows=2,
        cols=1,
        shared_xaxes=True,
        subplot_titles=[controller_config["base_trading_pair"], controller_config["quote_trading_pair"]],
        x_title="Time",
        y_title="Price"
    )

    # Add base market candlesticks
    fig.add_trace(
        go.Scatter(
            x=base_candles.index,
            y=base_candles["close"],
            mode="lines",
            name=f"{controller_config['base_trading_pair']} Close"
        ),
        row=1, col=1
    )

    # Add quote market candlesticks
    fig.add_trace(
        go.Scatter(
            x=quote_candles.index,
            y=quote_candles["close"],
            mode="lines",
            name=f"{controller_config['quote_trading_pair']} Close"
        ),
        row=2, col=1
    )

    # Add horizontal lines for the base market
    fig.add_hline(
        y=controller_config["grid_config_base"]["start_price"],
        row=1, col=1,
        line=dict(color="green", width=2)
    )
    fig.add_hline(
        y=controller_config["grid_config_base"]["end_price"],
        row=1, col=1,
        line=dict(color="green", width=2)
    )
    fig.add_hline(
        y=controller_config["grid_config_base"]["limit_price"],
        row=1, col=1,
        line=dict(color="green", dash="dash", width=2)
    )
    if plot_prices:
        # Add prices for base market
        base_prices, _ = await get_executor_prices(controller_config, connector_instance=connector_instance)
        for price in base_prices:
            fig.add_hline(
                y=price,
                row=1, col=1,
                line=dict(color="gray", dash="dash")
            )

        # Add prices for quote market
        quote_prices, _ = await get_executor_prices(controller_config, side="short", connector_instance=connector_instance)
        for price in quote_prices:
            fig.add_hline(
                y=price,
                row=2, col=1,
                line=dict(color="gray", dash="dash")
            )

    # Add horizontal lines for the quote market
    fig.add_hline(
        y=controller_config["grid_config_quote"]["start_price"],
        row=2, col=1,
        line=dict(color="red", width=2)
    )
    fig.add_hline(
        y=controller_config["grid_config_quote"]["end_price"],
        row=2, col=1,
        line=dict(color="red", width=2)
    )
    fig.add_hline(
        y=controller_config["grid_config_quote"]["limit_price"],
        row=2, col=1,
        line=dict(color="red", dash="dash", width=2)
    )
    fig.add_vline(pd.to_datetime(extra_info["timestamp"], unit="s"))

    # Update layout
    fig.update_layout(
        template="plotly_dark",
        xaxis_rangeslider_visible=False,
        xaxis2_rangeslider_visible=False,
        plot_bgcolor='rgba(0, 0, 0, 0)',
        paper_bgcolor='rgba(0, 0, 0, 0.1)',
        font={"color": 'white', "size": 12},
        height=400,  # Smaller height for thumbnails
        hovermode="x unified",
        showlegend=False
    )
    return fig


## Initialize Clients

In [ ]:
# Initialize clients
clob = CLOBDataSource()
mongo_client = MongoDBClient(
    username=os.getenv("MONGO_INITDB_ROOT_USERNAME", "admin"),
    password=os.getenv("MONGO_INITDB_ROOT_PASSWORD", "admin"),
    host=os.getenv("MONGO_HOST", "localhost"),
    port=os.getenv("MONGO_PORT", 27017),
    database=os.getenv("MONGO_DATABASE", "quants_lab")
)
connector_name = "binance_perpetual"
CONNECTOR_INSTANCE = clob.get_connector(connector_name)
await CONNECTOR_INSTANCE._update_trading_rules()

# Connect to MongoDB
await mongo_client.connect()

## Fetch Controller Configs and Candles

In [ ]:
# Parameters
days_to_download = 4
interval = "15m"
last_24h = time.time() - 1.5 * 24 * 60 * 60

# Fetch controller configs
controller_configs_data = await mongo_client.get_controller_config_data(min_timestamp=last_24h)

# Get exchange trading pairs
trading_rules = await clob.get_trading_rules(connector_name)
ex_trading_pairs = [trading_rule.trading_pair for trading_rule in trading_rules.data]

# Get trading pairs
base_trading_pairs = [config["config"]["base_trading_pair"] for config in controller_configs_data]
quote_trading_pairs = [config["config"]["quote_trading_pair"] for config in controller_configs_data]
trading_pairs = [trading_pair for trading_pair in list(set(base_trading_pairs + quote_trading_pairs)) if trading_pair in ex_trading_pairs]


# Fetch candles
candles_list = await clob.get_candles_batch_last_days(
    connector_name,
    trading_pairs,
    interval=interval,
    days=days_to_download,
    batch_size=60,
    sleep_time=2
)
candles_dict = {candle.trading_pair: candle.data for candle in candles_list}

## Apply Filters to Configs

In [ ]:
async def apply_filters(connector_instance,
                        config: Dict[str, Any],
                        base_candles: pd.DataFrame,
                        quote_candles: pd.DataFrame,
                        max_base_step: float,
                        max_quote_step: float,
                        min_grid_range_ratio: float,
                        max_grid_range_ratio: float,
                        max_entry_price_distance: float):
    # Calculate base grid metrics
    base_start_price = config["grid_config_base"]["start_price"]
    base_end_price = config["grid_config_base"]["end_price"]
    base_entry_price = base_candles["close"].iloc[-1]
    base_executor_prices, base_step = await get_executor_prices(config, connector_instance=connector_instance)

    base_grid_range_pct = base_end_price / base_start_price - 1
    base_entry_price_distance_from_start = (base_entry_price / base_start_price - 1) / base_grid_range_pct

    # Calculate quote grid metrics
    quote_start_price = config["grid_config_quote"]["start_price"]
    quote_end_price = config["grid_config_quote"]["end_price"]
    quote_entry_price = quote_candles["close"].iloc[-1]
    quote_executor_prices, quote_step = await get_executor_prices(config, side="short", connector_instance=connector_instance)

    quote_grid_range_pct = quote_end_price / quote_start_price - 1
    # todo
    quote_entry_price_distance_from_start = 1 - (quote_entry_price / quote_start_price - 1) / quote_grid_range_pct

    # Conditions using the input values
    base_step_condition = base_step <= max_base_step
    quote_step_condition = quote_step <= max_quote_step
    grid_range_gt_zero_condition = base_grid_range_pct > 0 and quote_grid_range_pct > 0
    grid_range_pct_condition = min_grid_range_ratio <= (
            base_grid_range_pct / quote_grid_range_pct) <= max_grid_range_ratio
    base_entry_price_condition = base_entry_price_distance_from_start < max_entry_price_distance
    quote_entry_price_condition = quote_entry_price_distance_from_start < max_entry_price_distance
    inside_grid_condition = ((base_start_price < base_entry_price < base_end_price) and
                             (quote_start_price < quote_entry_price < quote_end_price))
    price_non_zero_condition = (base_end_price > 0 and quote_end_price > 0 and base_start_price > 0
                                and quote_start_price > 0 and base_end_price > 0 and quote_end_price > 0)
    return (base_step_condition and quote_step_condition and grid_range_pct_condition and
            base_entry_price_condition and quote_entry_price_condition and inside_grid_condition
            and price_non_zero_condition and grid_range_gt_zero_condition)


def get_grid_executor_config(controller_config_dict: Dict[str, Any], side: str = "long"):
    side_config = "grid_config_base" if side == "long" else "grid_config_quote"
    return GridExecutorConfig(id=controller_config_dict["id"],
                              type="generic",
                              timestamp=time.time(),
                              leverage=controller_config_dict["leverage"],
                              connector_name=controller_config_dict["connector_name"],
                              trading_pair=controller_config_dict["base_trading_pair"],
                              start_price=controller_config_dict[side_config]["start_price"],
                              end_price=controller_config_dict[side_config]["end_price"],
                              limit_price=controller_config_dict[side_config]["limit_price"],
                              side=1 if side == "long" else 2,
                              total_amount_quote=controller_config_dict["total_amount_quote"] / 2,
                              min_spread_between_orders=controller_config_dict["min_spread_between_orders"],
                              min_order_amount_quote=controller_config_dict[side_config]["min_order_amount_quote"],
                              max_open_orders=controller_config_dict["max_open_orders"],
                              max_orders_per_batch=controller_config_dict["max_orders_per_batch"],
                              order_frequency=controller_config_dict[side_config]["order_frequency"],
                              activation_bounds=controller_config_dict["activation_bounds"],
                              triple_barrier_config=TripleBarrierConfig(
                                  stop_loss=controller_config_dict["triple_barrier_config"]["stop_loss"],
                                  take_profit=controller_config_dict["triple_barrier_config"]["take_profit"],
                                  time_limit=controller_config_dict["triple_barrier_config"]["time_limit"],
                                  trailing_stop=TrailingStop(
                                      activation_price=controller_config_dict["triple_barrier_config"]["trailing_stop"]["activation_price"],
                                      trailing_delta=controller_config_dict["triple_barrier_config"]["trailing_stop"]["trailing_delta"])))


async def get_executor_prices(executor_config_dict: Dict[str, Any],
                              connector_instance,
                              side: str = "long"):
    executor_config = get_grid_executor_config(executor_config_dict, side=side)
    connector_name = executor_config.connector_name
    with patch.object(connector_instance, "get_price_by_type", return_value=Decimal(str(executor_config.start_price))):
        controller = GridExecutor(ScriptStrategyBase({connector_name: connector_instance}), config=executor_config)
    prices = [level.price for level in controller.grid_levels]
    return prices, controller.step



# Filter parameters
filter_params = {
    "max_base_step": 0.001,
    "max_quote_step": 0.001,
    "min_grid_range_ratio": 0.5,
    "max_grid_range_ratio": 2.0,
    "max_entry_price_distance": 0.5
}

# Apply filters to configs
filtered_configs = []
for config_data in controller_configs_data:
    if config_data["config"]["base_trading_pair"] not in ex_trading_pairs:
        continue
    if config_data["config"]["quote_trading_pair"] not in ex_trading_pairs:
        continue
    config = config_data["config"]
    extra_info = config_data["extra_info"]
    base_candles = candles_dict[config["base_trading_pair"]]
    quote_candles = candles_dict[config["quote_trading_pair"]]
    
    try:
        meets_condition = await apply_filters(
            connector_instance=CONNECTOR_INSTANCE,
            config=config,
            base_candles=base_candles,
            quote_candles=quote_candles,
            **filter_params
        )
        if meets_condition:
            filtered_configs.append({
                "config": config,
                "base_candles": base_candles,
                "quote_candles": quote_candles,
                "extra_info": extra_info
            })
    except Exception as e:
        print(f"Error processing config: {e}")

print(f"Found {len(filtered_configs)} configs that meet the criteria")

## Visualize Filtered Configs

In [ ]:
import os
import plotly.graph_objects as go

# Ensure "img" directory exists
img_dir = "img"
os.makedirs(img_dir, exist_ok=True)

# Iterate over filtered configs and create figures
for i, config_data in enumerate(filtered_configs):
    # Generate the base figure
    fig = await create_coint_figure(
        connector_instance=CONNECTOR_INSTANCE,
        controller_config=config_data["config"],
        base_candles=config_data["base_candles"],
        quote_candles=config_data["quote_candles"],
        extra_info=config_data["extra_info"],
        plot_prices=False
    )

    # Extract details
    base_pair = config_data['config']['base_trading_pair']
    quote_pair = config_data['config']['quote_trading_pair']
    coint_value = config_data['extra_info']['coint_value']
    rate_diff = config_data['extra_info']['rate_difference']

    # Prepare annotation text
    info_text = (f"Config N°{i}<br>"
                 f"Base: {base_pair}<br>"
                 f"Quote: {quote_pair}<br>"
                 f"Coint Value: {coint_value:.3f}<br>"
                 f"Rate Diff: {rate_diff:.5f}%")

    # Define image filename
    img_filename = os.path.join(img_dir, f"config_{i:03d}.jpg")

    # Save the figure in high resolution
    fig.write_image(img_filename, format="jpg", scale=3)
print(f"Saved {len(filtered_configs)} images")

In [ ]:
selected_index = 1

selected_config_data = filtered_configs[selected_index]

config_to_deploy = selected_config_data["config"].copy()
extra_info = selected_config_data["extra_info"].copy()
base_trading_pair = config_to_deploy["base_trading_pair"]
quote_trading_pair = config_to_deploy["quote_trading_pair"]

if connector_name in ["okx_perpetual"]:
    min_notionals_dict = {trading_rule.trading_pair: float(trading_rule.min_base_amount_increment) * candles_dict[trading_rule.trading_pair].close.iloc[-1] for trading_rule in trading_rules.data if candles_dict.get(trading_rule.trading_pair) is not None}
else:
    min_notionals_dict = {trading_rule.trading_pair: trading_rule.min_notional_size for trading_rule in trading_rules.data}

config_to_deploy

In [ ]:
total_amount_quote = 1000.0
min_spread_between_orders = 0.0004
base_min_order_amount_quote = float(min_notionals_dict[base_trading_pair]) * 1.5
quote_min_order_amount_quote = float(min_notionals_dict[quote_trading_pair]) * 1.5
leverage = 50
time_limit = 259200
stop_loss = 0.1
trailing_delta = 0.005
take_profit = 0.008
activation_price = 0.03

now = datetime.now()
year, iso_week, _ = now.isocalendar()
formatted = f"{year}||isoweek{iso_week}"
controller_id = f"{connector_name}||{config_to_deploy['base_trading_pair']}||{config_to_deploy['quote_trading_pair']}||{formatted}"
tag = "01"

In [ ]:
min_order_amount_quote = max(base_min_order_amount_quote, quote_min_order_amount_quote)
config_to_deploy["id"] = f"{controller_id}_{tag}"
config_to_deploy["total_amount_quote"] = total_amount_quote
config_to_deploy["coerce_tp_to_step"] = True
config_to_deploy["grid_config_base"]["min_order_amount_quote"] = min_order_amount_quote
config_to_deploy["grid_config_quote"]["min_order_amount_quote"] = min_order_amount_quote
config_to_deploy["leverage"] = leverage
config_to_deploy["connector_name"] = connector_name
config_to_deploy["min_spread_between_orders"] = min_spread_between_orders / 100
config_to_deploy["triple_barrier_config"] = {
    'stop_loss': stop_loss,
    'take_profit': take_profit,
    'time_limit': time_limit,
    'trailing_stop': {'activation_price': activation_price, 'trailing_delta': trailing_delta}
}

In [ ]:
base_candles = candles_dict[base_trading_pair]
quote_candles = candles_dict[quote_trading_pair]

# Create detailed figure
detailed_fig = await create_coint_figure(CONNECTOR_INSTANCE, config_to_deploy, base_candles, quote_candles, extra_info, plot_prices=True)
detailed_fig.update_layout(height=800)

In [ ]:
config_to_deploy

In [ ]:
# backend_api_client = BackendAPIClient(host=os.getenv("BACKEND_API_SERVER"))
backend_api_client = BackendAPIClient(host=os.getenv("BACKEND_API_SERVER", "localhost"))

await backend_api_client.add_controller_config(config_to_deploy)

In [ ]:
top_configs = [config_to_deploy]
await backend_api_client.deploy_script_with_controllers(bot_name=f"{connector_name}-ts-{iso_week}-{tag}",
                                                        controller_configs=[config["id"] + ".yml" for config in top_configs],
                                                        script_name="v2_with_controllers.py",
                                                        image_name="hummingbot/hummingbot:latest",
                                                        credentials=os.getenv("DEPLOY_CREDENTIALS", "master_account"),
                                                        time_to_cash_out=None,
                                                        max_global_drawdown=None,
                                                        max_controller_drawdown=None)